In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import pickle
from numba import njit, prange
import psycopg2
import psycopg2.extras
from psycopg2 import sql
from itertools import product
from datetime import timedelta, datetime
from skimage.filters import threshold_minimum, threshold_mean
from scipy import stats
from scipy.stats import norm
import matplotlib.dates as mdates
from tqdm import tqdm_notebook
import h5py
import gc

from distributed import worker_client

import bb_utils
import bb_utils.meta
from bb_utils.ids import BeesbookID
import bb_backend
from bb_backend.api import FramePlotter, VideoPlotter
from bb_backend.api import get_plot_coordinates, transform_axis_coordinates, get_image_origin

bb_backend.api.server_adress = '127.0.0.1:8000'

sns.set()

/usr/local/lib/python3.5/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [3]:
import seaborn as sns
import matplotlib
def setSnsStyle(style):
    sns.set(style=style, font_scale=1.5)
    font = {'family' : 'serif',
       'weight' : 'normal',
       'size'   : 16}
    matplotlib.rc('font', **font)
    matplotlib.rcParams['xtick.labelsize'] = 16
    matplotlib.rcParams['ytick.labelsize'] = 16
    matplotlib.rcParams['axes.titlesize'] = 16
    matplotlib.rcParams['axes.labelsize'] = 16
   
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
#%config InlineBackend.figure_format = 'retina'

setSnsStyle("ticks")

In [4]:
connect_str = """dbname='beesbook' 
                 user='reader' 
                 host='tonic.imp.fu-berlin.de' 
                 password='' 
                 application_name='sammelgruppen'"""

In [5]:
import os
import dask.distributed
from distributed.security import Security

sec = Security(tls_ca_file='/home/janek/ssl-cert/tls-ca-cert.pem',
               tls_client_cert='/home/janek/ssl-cert/tls-cert.pem',
               tls_client_key='/home/janek/ssl-cert/tls-key.pem',
               require_encryption=True)

client = dask.distributed.Client(address='tls://tonic.imp.fu-berlin.de:9733', 
                                 timeout=120,
                                 security=sec)

client

FileNotFoundError: [Errno 2] No such file or directory

In [ ]:
db_workers = set([w for w in list(client.ncores()) if '160.45.109.25' in w])
compute_workers = set([w for w in list(client.ncores()) if w not in db_workers])

In [ ]:
meta = bb_utils.meta.BeeMetaInfo()

In [ ]:
def transform_axis_coordinates(x, y, cam_id=None, year=2016):
    origin = get_image_origin(cam_id, year)
    if origin[0] == 1:
        x = 3000 - x
    if origin[1] == 0:
        y = 4000 - y
    return x, y

In [ ]:
num_days = 1

In [ ]:
group_idx = 20

group_ids = map(lambda i: i.as_ferwar(), 
                list(map(bb_utils.ids.BeesbookID.from_dec_12, meta.get_foragergroup(group_idx).dec12)))
group_ids = list(group_ids)

In [ ]:
group_ids

In [ ]:
group = meta.get_foragergroup(group_idx)

In [ ]:
[(i, meta.get_beename(BeesbookID.from_dec_12(i))) for i in meta.get_foragergroup(group_idx).dec12]

In [ ]:
def get_ids(end_date, dt, hour):
    with psycopg2.connect(connect_str) as conn:
        ids = pd.read_sql_query(
                """SELECT bee_id, COUNT(*) from bb_detections 
                   WHERE
                       timestamp >= %s
                       AND timestamp <= %s
                   GROUP BY bee_id;""", conn, 
                params=(end_date - timedelta(days=dt) + timedelta(hours=hour), 
                        end_date - timedelta(days=dt) + timedelta(hours=hour+1)))
    return ids

In [ ]:
end_date = group.date + timedelta(days=1)
all_ids = []
futures = []

In [ ]:
for dt in list(range(num_days, 0, -1)):
    print('dt = ', dt)
    for hour in range(24):
        futures.append(client.submit(get_ids, end_date, dt, hour, workers=db_workers, retries=5))

for future in tqdm_notebook(futures):
    all_ids.append(client.gather([future])[0])

In [ ]:
client.cancel(futures)

In [ ]:
merged_df = all_ids[0]
for df in all_ids[1:]:
    merged_df = pd.merge(merged_df, df, how='outer', on=['bee_id']).set_index(['bee_id']).sum(axis=1)
    merged_df = merged_df.reset_index(name='count')

In [ ]:
sns.distplot(merged_df['count'])

In [ ]:
sns.distplot(np.log(merged_df['count']), bins=25)